In [ ]:
import math
import random

def is_prime(num):
    factor = 2
    while factor * factor <= num:
        if num % factor == 0:
             return False
        factor +=1
    return True


def nth_prime_number(n):
    if n==1:
        return 2
    count = 1
    num = 1
    while(count < n):
        num +=2 #optimization
        if is_prime(num):
            count +=1
    return num

def permutation_table(n):
    perm = []
    inv_perm = []
    for i in range(n):
        perm.append(i)
        inv_perm.append(i)

    random.shuffle(perm)
    for i in range(n):
        inv_perm[perm[i]] = i
    return perm, inv_perm

def radical_inverse(base, a, perm=None):
    inv_base = 1.0 / base
    reversed_digits = 0
    inv_base_n = 1.0

    while True:
        next_ = a // base
        digit = a - next_ * base

        # apply permutation if required
        if perm is not None:
            assert(perm[digit] < base)
            digit = perm[digit]

        reversed_digits = reversed_digits * base + digit
        inv_base_n *= inv_base
        a = next_

        if  a <= 0:
            break

    inverse = inv_base_n * reversed_digits
    assert(inverse < 1.00001);
    return inverse

def inverse_radical_inverse(base, inverse, num_digits, inv_perm=None):
    index = 0
    while num_digits > 0:
        digit = inverse % base;
        if inv_perm is not None:
            digit = inv_perm[digit]

        inverse = inverse // base
        index = index * base + digit
        num_digits = num_digits - 1

    return index


In [ ]:
for n in range(1, 20):
    base = nth_prime_number(n)
    perm, inv_perm = permutation_table(base)
    for a in range(100):
        radical_inverse(base, a, perm)

print("done")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=[4,4])

ndim = 1
perturb = True
perm, inv_perm = None, None

p = [[],[]]
for dim in range(2):
    base = nth_prime_number(ndim + dim)
    if perturb:
        perm, inv_perm = permutation_table(base)

    for i in range(256):
        p[dim].append(radical_inverse(base, i, perm))

plt.scatter(p[0], p[1], color='black', s=2)
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

In [ ]:
def next_power_of_base(base, i):
    if i == 0:
        return 1, 0

    n = base
    exp = 1
    while n <= i:
        n *= base
        exp += 1
    return n, exp

for dim in range(1, 200):
    base = nth_prime_number(dim)
    perm, inv_perm = permutation_table(base)
    for a in range(20000):
        s = radical_inverse(base, a, perm)
        n, n_digits = next_power_of_base(base, a)
        inverse = round(s * n)
        a_ = inverse_radical_inverse(base, inverse, n_digits, inv_perm)
        if a_ - a != 0:
            print("error: ", "a", a, "a_", a_)

print("done")


In [ ]:
import numpy as np

def chinese_remainder(n, a):
    sum=0
    prod=reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n,a):
        p=prod//n_i
        sum += a_i* mul_inv(p, n_i)*p
    return sum % prod
def mul_inv(a, b):
    b0= b
    x0, x1= 0,1
    if b== 1: return 1
    while a>1 :
        q=a// b
        a, b= b, a%b
        x0, x1=x1 -q *x0, x0
    if x1<0 : x1+= b0
    return x1
n=[3,5,7]
a=[2,3,2]
v = chinese_remainder(n,a)

print(np.remainder(v, n))

In [ ]:
import math
import numpy as np
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from functools import reduce


def dither17(x, y):
    FrameIndexMod4 = 0
    v = np.dot(np.array([x, y, FrameIndexMod4]), np.array([2,7,23])/17)
    return math.fmod(v, 1.0)

def ign(x, y):
    return math.fmod(52.9829189 * math.fmod(0.06711056*x + 0.00583715*y, 1.0), 1.0)

pow = np.zeros(2)
exp = np.zeros(2)
for i in [0,1]:
    pow[i], exp[i] = next_power_of_base(nth_prime_number(i + 1), rez-1)

perm, inv_perm = None, None

def pixel(s, t, rez):
    # return dither17(s, t)
#     return ign(s,t)
#     return (s + t * rez + np.random.rand(1)) / (rez**2)
#     i = t * rez + s
#     return math.fmod(.618033988749894 * i, 1)
    # return math.fmod(n, 1)
    # return np.random.rand(1)
    idx = np.zeros(2)
    for i in [0,1]:
        prime = nth_prime_number(i + 1)
        a = s if i == 0 else t
        idx[i] = inverse_radical_inverse(prime, a, exp[i], inv_perm)

        # sanity check
        a_ = np.round(radical_inverse(prime, idx[i], perm) * pow[i])
        assert a == a_

    a = int(chinese_remainder(pow, idx))
    # a = t * rez + s
    dim = 3
    base = nth_prime_number(dim)
    return radical_inverse(base, a)

rez = 128
bitmap = np.zeros((rez,rez))
for t in range(rez):
    for s in range(rez):
        bitmap[t][s] = pixel(s, t, rez)

plt.figure(figsize=[6,6])
plt.imshow(bitmap, cmap=cm.gray)
plt.show()


freq = np.fft.fft2(bitmap).real
freq = np.abs(freq)

plt.figure(figsize=[6,6])
# plt.imshow(freq, norm=LogNorm(vmin=5))
plt.imshow(freq, norm=LogNorm(vmin=2), cmap=cm.gray)
plt.show()


In [ ]:
import ipyvolume as ipv


x=np.arange(rez)
y=np.arange(rez)
x,y = np.meshgrid(x, y)
ipv.clear()
ipv.plot_surface(x,y,freq)
ipv.zlim(0,800)
ipv.show()
